In [1]:
# from tensorflow.keras import backend
from tensorflow.keras.models import load_model
from unet3 import *
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"  # Only GPU 3 is visible to this code

# load json and create model 
json_file = open('model/model3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
# loaded_model.load_weights("model/pretrained_model.hdf5")
# loaded_model.load_weights("model/pretrained_model.hdf5")

print("Loaded model from disk")

# training image dimensions
n1, n2, n3 = 128, 128, 128

/home/anyu/anaconda3/envs/faultSeg36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anyu/anaconda3/envs/faultSeg36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anyu/anaconda3/envs/faultSeg36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anyu/anaconda3/envs/faultSeg36/lib

Loaded model from disk


In [40]:
data_folder = "..<DATA FOLDER>.."
# load validation set for prepare 3d cube, can change to training set to prepare corresponding 3D cube training set
seis = np.load("{}/thebeorigin/seisval.npy").format(data_folder)
fault = np.load("{}/thebeorigin/faultval.npy").format(data_folder)
print(seis.shape, fault.shape)

gx = seis.astype(np.single)
gy = fault.astype(np.single)
print(gx.shape, gx.dtype)
print(gy.shape, gy.dtype)

m1,m2,m3 = 200,3174,1537
print(m1,m2,m3 )

(200, 3174, 1537) (200, 3174, 1537)
(200, 3174, 1537) float32
(200, 3174, 1537) float32
200 3174 1537


In [41]:
import matplotlib.pyplot as plt
%matplotlib inline

#a 3d array of gx[m1][m2][m3], please make sure the dimensions are correct!!!
#we strongly suggest to gain the seismic image before input it to the faultSeg!!!
# gx,m1,m2,m3 = np.fromfile("/data/anyu/faultSegData/prediction/f3d/gxl.dat",dtype=np.single),512,384,128
os = 64 #overlap width
c1 = np.round((m1+os)/(n1-os)+0.5)
c2 = np.round((m2+os)/(n2-os)+0.5)
c3 = np.round((m3+os)/(n3-os)+0.5)
print(c1,c2,c3)
c1 = int(c1)
c2 = int(c2)
c3 = int(c3)
p1 = (n1-os)*c1+os
p2 = (n2-os)*c2+os
p3 = (n3-os)*c3+os
print(c1,c2,c3)
print(p1,p2,p3)

5.0 51.0 26.0
5 51 26
384 3328 1728


In [42]:
pixelThre = int(0.03*128*128*128)
print(pixelThre)

62914


In [43]:
count = 0
gx = np.reshape(gx,(m1,m2,m3))
gy = np.reshape(gy,(m1,m2,m3))

gp = np.zeros((p1,p2,p3),dtype=np.single)
gp1 = np.zeros((p1,p2,p3),dtype=np.single)

gs = np.zeros((1,n1,n2,n3,1),dtype=np.single)
gs1 = np.zeros((1,n1,n2,n3,1),dtype=np.single)

gp[0:m1,0:m2,0:m3]=gx
gp1[0:m1,0:m2,0:m3]=gy

for k1 in range(c1):
    for k2 in range(c2):
        for k3 in range(c3):
            b1 = k1*n1-k1*os
            e1 = b1+n1
            b2 = k2*n2-k2*os
            e2 = b2+n2
            b3 = k3*n3-k3*os
            e3 = b3+n3
            gs[0,:,:,:,0]=gp[b1:e1,b2:e2,b3:e3]
            gs1[0,:,:,:,0]=gp1[b1:e1,b2:e2,b3:e3]
            if gs1.sum() > pixelThre:
                gs.flatten().tofile("{}/faultSegData/thebe3dcube/validation/seis/"+"{}.dat".format(data_folder,count),format="%4")
                gs1.flatten().tofile("{}/faultSegData/thebe3dcube/validation/fault/"+"{}.dat".format(data_folder,count),format="%4")
                count = count+1
print(count)

631


In [30]:
print(gs.flatten().shape,gs.sum())

(2097152,) 0.0


In [31]:
# gs.tofile("test"+"gs.dat",format="%4")

In [6]:
# check if it is successfully stored.
gx = np.fromfile("{}/faultSegData/train/seis/8.dat".format(data_folder),dtype=np.single)
print(gx.shape,gx.dtype,gx.min(),gx.max())

(2097152,) float32 -7.59338 7.2299137
